# Report

## Introduction and data

> REMOVE THE FOLLOWING TEXT

This section includes an introduction to the project motivation, data, and research question.
Describe the data and definitions of key variables.

It should also include some exploratory data analysis.

All of the EDA won't fit in the paper, so focus on the EDA for the response variable and a few other interesting variables and relationships.

## Die Bedeutung von YouTube für Nachrichtenmedien

YouTube hat sich in den letzten Jahren zu einer der wichtigsten Plattformen für den Konsum von Videoinhalten entwickelt. Insbesondere für Nachrichtenmedien wie ntv ist die Plattform zu einem unverzichtbaren Kanal geworden, um ihre Zielgruppe zu erreichen. Die Analyse der Performance von YouTube-Videos und das Verständnis der Faktoren, die zum Erfolg einzelner Beiträge führen, sind dabei von entscheidender Bedeutung für die strategische Ausrichtung des Kanals.

Für Nachrichtenmedien stellt sich dabei die besondere Herausforderung, dass sie einerseits journalistischen Qualitätsstandards gerecht werden müssen, andererseits aber auch die Mechanismen der Plattform verstehen und nutzen müssen, um ihre Inhalte erfolgreich zu platzieren. Die vorliegende Analyse soll dazu beitragen, diese Balance besser zu verstehen und zu optimieren.

## Forschungsfrage und Hypothesen

Die zentrale Forschungsfrage dieser Analyse lautet: Welche Faktoren beeinflussen die Performance von Videos auf dem ntv YouTube-Kanal und wie können diese Erkenntnisse genutzt werden, um die Content-Strategie zu optimieren?

Dabei werden drei zentrale Hypothesen untersucht: Erstens wird angenommen, dass Videos zu bestimmten Themengebieten wie dem Ukrainekrieg oder Bitcoin im Durchschnitt mehr Aufrufe und eine längere Wiedergabedauer erzielen als Videos zu anderen Themen. Zweitens wird vermutet, dass die Art der Thumbnail-Gestaltung, insbesondere die prominente Platzierung bekannter Experten, einen signifikanten Einfluss auf die Klickrate hat. Die dritte Hypothese besagt, dass eine höhere SEO-Bewertung des Videotitels positiv mit der Anzahl der Aufrufe und der Wiedergabedauer korreliert.

## Datengrundlage und Erhebungsprozess

Der analysierte Datensatz umfasst etwa 2500 Videos, die im Jahr 2024 auf dem ntv YouTube-Kanal veröffentlicht wurden. Die Datenerhebung gestaltete sich dabei als komplexer Prozess, der verschiedene Quellen und Methoden kombiniert. Während grundlegende Metriken wie Aufrufe, Likes und Kommentare über die YouTube Analytics API bezogen werden können, sind spezielle Metriken wie die Klickrate der Impressionen nur durch manuelle Extraktion aus dem YouTube Studio Backend verfügbar. Dies macht die Datensammlung zu einem aufwändigen Prozess, der sowohl automatisierte als auch manuelle Schritte erfordert.

Zusätzlich zu den von YouTube bereitgestellten Metriken wurden weitere Kategorisierungen und Bewertungen vorgenommen. Die Videos wurden nach ihren Themenbereichen kategorisiert (Politik, Wirtschaft, Krieg, Sonstiges, Bilder, Live). Die Thumbnail-Gestaltung wurde nach einem dreistufigen System bewertet, das insbesondere die Präsenz und Positionierung wichtiger Persönlichkeiten berücksichtigt. Die Kategorie "3" gab es bei Thumbnails die eine Kombination von Gesicht und Titel besonders präsent ist. Die Kategorie "2" gab es, wenn eines von beiden vorhanden war. Entweder ein normales Bild + präsenten Tiel oder präsentes Gesicht. Eine "1" wurde für Thumbnails vergeben, die keine besonderen Merkmale aufgewiesen haben. In der Datei "youtube Daten abrufen Test2.ipynb" ist auch ein Code abgelegt, der über "Google Vision AI" eine Bewertung der Thumnbails nach diesen Kategorien erfolgen soll. Leider war hier die Parameter zu ungenau. Eine Verbesserung des systems wäre für das Projekt zu aufwendig und teuer gewesen. Hier müsste noch mehr Aufwand betrieben werden oder ein eigenes System trainiert werden. Das wäre die Aufgabe für ein neues Projekt. 

Für die Bewertung der Videotitel wurde ein KI-gestütztes System entwickelt, das die Titel nach SEO-Kriterien auf einer Skala von 0.0 bis 10,0 bewertet. Dabei wurden durch die KI Bewertungen zwischen 4,5 und 8,5 vorgenommen. Als KI wurde ChatGPT per API genutzt. Der Code und auch der Prompt dafür ist in der Datei youtube Daten abrufen Test2.ipynb abgelegt. 

## Zentrale Metriken und ihre Bedeutung für einen Nachrichtenkanal

Für die Analyse wurden drei zentrale Zielvariablen identifiziert, die für Nachrichtenmedien auf YouTube von besonderer Bedeutung sind:

Die durchschnittliche Wiedergabedauer ist besonders für Nachrichtenformate relevant, da sie Aufschluss darüber gibt, wie gut es gelingt, komplexe Sachverhalte für die YouTube-Zielgruppe aufzubereiten. Anders als bei traditionellen TV-Nachrichten, wo Zuschauer tendenziell das gesamte Format konsumieren, ist die Haltedauer auf YouTube ein kritischer Erfolgsfaktor. Sie beeinflusst direkt, wie häufig ein Video vom Algorithmus anderen Nutzern vorgeschlagen wird. So zeigen erfolgreiche Videos des ntv-Kanals, dass besonders Live-Berichterstattungen zu wichtigen Ereignissen, wie beispielsweise die Iran-Angriffe auf Israel, überdurchschnittliche Wiedergabedauern erzielen. Bei diesen Events bleiben Zuschauer oft über 10 Minuten am Video, während der Durchschnitt bei Nachrichtenvideos bei etwa 4-5 Minuten liegt.

Die Anzahl der Aufrufe ist für Nachrichtenkanäle nicht nur eine reine Reichweitenkennzahl, sondern auch ein Indikator für die gesellschaftliche Relevanz ihrer Berichterstattung. Im hart umkämpften Nachrichtenmarkt auf YouTube, wo etablierte Medien mit einer Vielzahl alternativer Informationsquellen konkurrieren, sind hohe Aufrufzahlen zudem entscheidend für die Monetarisierung und damit die Finanzierung hochwertiger journalistischer Arbeit. Die Analyse zeigt, dass besonders Videos mit prominenten Wirtschaftsexperten wie Hans-Werner Sinn oder Politikern wie Robert Habeck regelmäßig überdurchschnittliche Aufrufzahlen generieren. Dabei spielen sowohl die Bekanntheit der Personen als auch die Aktualität und Relevanz der Themen eine entscheidende Rolle.

Die Klickrate der Impressionen hat für Nachrichtenmedien eine besondere strategische Bedeutung. Sie zeigt, wie gut es gelingt, in der Informationsflut von YouTube die Aufmerksamkeit potenzieller Zuschauer zu gewinnen, ohne dabei in reißerische oder irreführende Darstellungen zu verfallen. Erfolgreiche Beispiele aus dem ntv-Kanal demonstrieren, dass Videos mit klar erkennbaren Experten im Thumbnail und prägnanten, aber sachlichen Titeln Klickraten von über 10% erreichen können - deutlich über dem Durchschnitt von etwa 4-5%. Besonders erfolgreich sind dabei Thumbnails, die bekannte Persönlichkeiten in emotionalen Momenten oder bei markanten Aussagen zeigen, kombiniert mit kurzen, prägnanten Headlines.

Zusammen bilden diese drei Metriken ein komplexes Erfolgssystem: Ein Paradebeispiel sind die Live-Übertragungen wichtiger politischer Ereignisse. Diese erzielen oft hohe initiale Klickraten durch zeitkritische, aber seriöse Thumbnails (etwa "LIVE: Scholz gibt Regierungserklärung ab"), halten die Zuschauer durch die Aktualität und Relevanz des Geschehens lange im Video und erreichen durch diese Kombination hohe Gesamtaufrufzahlen. Ein weiteres erfolgreiches Format sind Wirtschaftsanalysen mit bekannten Experten, die durch ihre Expertise sowohl hohe Klickraten als auch überdurchschnittliche Wiedergabedauern erzielen.

Diese Best Practices zeigen, dass journalistische Qualität und YouTube-Optimierung keinen Widerspruch darstellen müssen. Vielmehr geht es darum, qualitativ hochwertige Inhalte so zu präsentieren, dass sie die Mechanismen der Plattform optimal nutzen. Die Herausforderung liegt dabei in der kontinuierlichen Anpassung an sich verändernde Algoritmen und Nutzergewohnheiten, ohne die journalistischen Grundprinzipien zu vernachlässigen.


## Limitationen und Herausforderungen

Bei der Analyse müssen einige wichtige Einschränkungen berücksichtigt werden. Die teilweise notwendige manuelle Datenextraktion erhöht das Risiko von Fehlern und macht den Prozess zeitaufwändig. Die Kategorisierung der Themen und die Bewertung der Thumbnails enthalten zwangsläufig subjektive Elemente, auch wenn versucht wurde, diese durch klare Kriterien zu minimieren. Zudem spielen externe Faktoren wie aktuelle Nachrichtenereignisse, Änderungen im YouTube-Algorithmus und saisonale Effekte eine wichtige Rolle, die bei der Interpretation der Ergebnisse berücksichtigt werden müssen.

Für die weitere Bewertung der Analysen ist es noch einmal wichtig, die grundlegenden Kennziffern und deren Bedeutung zu erfassen. Hier liegt der Schlüssel für den Erfolg einer möglichen Analyse. Ich beschränke mich auf die drei wichtigen Erfolgskennzahlen: Views, Impressions und Klickrate der Impressions. Die Wiedergabedauer ist noch eine wichtige Zahl, muss hier aber nicht näher erläutert werden. 

Im YouTube-Universum sind die Begriffe „Impressions“, „Klickrate der Impressions“ und „Views“ zentrale Kennzahlen, die häufig für die Performance-Analyse eines Channels herangezogen werden. Hier eine kurze Erklärung:

##### 1. Impressions (Sichtkontakte)

- **Was bedeutet das?**  
  Impressions geben an, wie oft ein Thumbnail (Vorschaubild) eines Videos auf YouTube den Nutzer*innen angezeigt wurde. Dabei spielt es keine Rolle, ob das Video letztendlich angeklickt und angesehen wird – jede einzelne Einblendung des Thumbnails in den Suchergebnissen, den Empfehlungen oder auf anderen YouTube-Oberflächen zählt als Impression.

- **Wozu dient diese Kennzahl?**  
  \- Sie zeigt das Potenzial für Reichweite: Wie viele Leute _könnten_ das Video sehen (und ggf. anklicken)?  
  \- Je höher die Impressions, desto mehr Nutzer*innen wurde das Video theoretisch präsentiert.

---

##### 2. Klickrate der Impressions (Impression Click-Through-Rate, CTR)

- **Was bedeutet das?**  
  Die Klickrate (CTR) ist der prozentuale Anteil der Impressions, bei denen tatsächlich auf das Video geklickt wurde. Anders ausgedrückt: Wenn dein Thumbnail 100 Mal angezeigt wird und 10 Mal geklickt wird, beträgt die Klickrate 10 %.  

  \[
  \text{Klickrate der Impressions} = \frac{\text{Klicks auf das Video}}{\text{Impressions}} \times 100\% 
  \]

- **Wozu dient diese Kennzahl?**  
  \- Sie ist ein guter Indikator dafür, wie „anziehend“ der Titel und das Thumbnail deines Videos sind.  
  \- Eine niedrige Klickrate kann bedeuten, dass du das Video zwar oft ausgespielt bekommst (also viele Impressions), aber die Nutzer*innen sich nicht für den Klick entscheiden.  
  \- Eine höhere Klickrate deutet darauf hin, dass das Vorschaubild und der Titel des Videos besonders ansprechend sind und Nutzer*innen zum Anschauen animieren.

---

##### 3. Views (Aufrufe)

- **Was bedeutet das?**  
  Views zählen, wie oft ein Video tatsächlich angesehen wurde – also wie oft jemand den Abspielknopf drückt (und das Video mindestens für eine gewisse Zeit ansieht). Die konkrete Definition, wann ein „View“ gezählt wird, hängt von YouTube-internen Kriterien ab (z. B. Mindestwiedergabedauer), sodass nicht sofort jeder Aufruf in Echtzeit als View erscheint.

- **Wozu dient diese Kennzahl?**  
  \- Views sind wohl das bekannteste Maß für den Erfolg eines Videos.  
  \- Zeigt dir, wie oft dein Content tatsächlich konsumiert wurde.  
  \- Zusammen mit weiteren Daten (z. B. Wiedergabedauer, Zuschauerbindung) bekommst du ein Bild davon, wie interessant die Inhalte sind und wie lange die Zuschauer*innen dabeibleiben.

---

##### Wichtig zu wissen
- **Zusammenspiel der Kennzahlen**:  
  \- Hohe Impressions, aber niedrige Klickrate = Zeichen, dass das Video zwar vielen Nutzer*innen angezeigt wird, aber das Thumbnail/Titel nicht zum Klicken motiviert.  
  \- Niedrige Impressions, aber hohe Klickrate = Zeichen, dass das Video in der Zielgruppe, in der es ausgespielt wird, gut ankommt, aber insgesamt noch nicht genug Reichweite erzielt.  
  \- Views und Klickrate hängen indirekt zusammen. Eine höhere Klickrate führt in der Regel zu mehr Views – sofern YouTube das Video möglichst oft als Impression ausspielt.

- **Video-Optimierung**:  
  \- Experimentiere mit Thumbnails, Titel und Beschreibungen, um eine gute Balance zwischen **vielen Impressions** und einer **hohen Klickrate** zu erreichen.  
  \- Eine gute Zuschauerbindung (Retention) ist ebenfalls wichtig, weil YouTube-Algorithmen Videos mit hoher Zuschauerzufriedenheit häufiger vorschlagen.

- **Gerätenutzung**:    
  \- Für die Bewertung der Ergebnisse könnte auch wichtig sein, auf welchen Geräten die User die Youtube-Vides schauen:

    <span style="color: red; font-weight: bold;">  Handy: 61,9 %, Computer: 15,6 %, TV: 13,6 %, Tablet: 8,8 % </span>

  \- Oder das Geschlecht und Alter der User:

    <span style="color: red; font-weight: bold;">  Männlich 80%, Alter: 55+ 61 %, 25 bis 55: 35 % </span>

---

**Kurzfassung**  
- **Impressions**: Wie oft wird das Video-Thumbnail den Nutzer*innen angezeigt?  
- **Klickrate (CTR)**: Wie groß ist der prozentuale Anteil, der von den Impressions tatsächlich zum Klick führt?  
- **Views**: Wie oft wurde das Video tatsächlich angesehen?

**Schlussfolgerungen der bisherigen Datenanalyse und Datenvorverarbeitung**


#### **1. Datenstruktur und Vorbereitung**
- Der Datensatz enthält sowohl numerische als auch kategoriale Merkmale.
- Die Zielvariable **"Aufrufe"** zeigt eine starke Rechtsschiefe mit vielen niedrigen und wenigen extrem hohen Werten.
- Merkmale wie **"video_id", "title", "publish_date"** wurden als Identifikationsspalten ausgeschlossen, da sie keinen direkten Einfluss auf die Vorhersage der Aufrufzahlen haben. 

#### **2. Bewertung der Variablen**
- **Bewertung_Titel:**
  - Diese Variable wurde auf einer Skala von 0 bis 10 vergeben und sollte daher nicht skaliert werden, um die interpretierbare Einteilung (z. B. schlechte, durchschnittliche und gute Bewertungen) beizubehalten.
  - Die Verteilung der Werte zeigte eine starke Konzentration auf einige wenige Werte. Die reine SEO-Bewertung scheint nicht sinnvoll, da die KollegInnen bereits auf einem hohen Noveau arbeiten und die Bewertung zwischen 6 und 8 laufen. 

- **Gestaltung_Thumbnail:**
  - Diese Variable wurde in den Werten 1, 2 und 3 vergeben und spiegelt eine Einteilung in Kategorien wider (z. B. schlecht, neutral, gut).
  - Auch hier wurde festgestellt, dass eine Standardisierung nicht sinnvoll ist, da es sich um feste Kategorien handelt.

#### **3. Erforderlichkeit der Skalierung**
- Bei den numerischen Variablen mit hohen Wertebereichen wurde festgestellt, dass eine Skalierung sinnvoll ist, um Unterschiede in den Daten nicht zu stark zu gewichten:
  - **Spalten mit großen Wertebereichen:**
    - `video_length_seconds` (Werte von 61 bis 39.061 Sekunden).
    - `wiedergabezeit_minuten`, `aufrufe`, `likes`, `dislikes`, `kommentare`, `Impressionen`.
  - Diese Spalten wurden für eine Standardisierung identifiziert, um die Modelle nicht durch stark variierende Wertebereiche zu verzerren.
  - Nach einer ersten Voranalyse der Daten wurde am Ende auf den Scaler verzichtet.

- **Keine Skalierung erforderlich bei:**
  - **`Bewertung_Titel`**: Da diese Werte auf einer vordefinierten Skala basieren (0 bis 10).
  - **`Gestaltung_Thumbnail`**: Diese Werte sind ordinal und haben eine konkrete Bedeutung.

#### **4. Zusammenhänge zwischen Prädiktoren und Zielvariable**
- **Themenzugehörigkeit:**
  - Das Thema `Krieg` zeigte einen deutlichen Einfluss auf die Aufrufzahlen. Videos mit diesem Thema hatten im Vergleich zu anderen Themen höhere Median-Aufrufzahlen und eine breite Streuung.
  - Im Gegensatz dazu zeigte das Thema `Politik` keinen signifikanten Unterschied zwischen Videos mit und ohne diese Zuordnung.

- **Bewertung_Titel und Gestaltung_Thumbnail:**
  - In den Streudiagrammen wurde kein starker Zusammenhang zwischen der Bewertung der Titel und der Aufrufzahl festgestellt.
  - Eine Analyse der Verteilung zeigte, dass die Bewertungen stark auf wenige Werte konzentriert waren (z. B. viele neutrale und wenige sehr gute oder sehr schlechte Bewertungen), was den fehlenden Zusammenhang erklären könnte. Hier müsste eine  Gruppierung über Kategorien oder auch Datum erfolgen, um eventuell einen Zusammenhang herzustellen. Eventuell auch einmal alle Extremwerte aus den Daten entfernen. 

#### **5. Empfehlungen für die Datenvorverarbeitung und Modellierung**
- **Skalierung:** Führe eine selektive Standardisierung der numerischen Spalten mit hohen Wertebereichen durch.
- **Feature-Engineering:** Prüfe, ob Kategorien wie `Gestaltung_Thumbnail` besser durch One-Hot-Encoding dargestellt werden sollten.
- **Logarithmische Transformation:** Bei der Zielvariable `aufrufe` könnte eine Log-Transformation sinnvoll sein, um die rechtsschiefe Verteilung auszugleichen und Ausreißer abzumildern.
- **Mehrfaktorenanalyse:** Kombinationen von Prädiktoren (z. B. `Gestaltung_Thumbnail` und `Bewertung_Titel`) sollten in Bezug auf die Aufrufe untersucht werden.

### **Erste Zusammenfassung:**
Die bisherige Analyse hat gezeigt, dass die Merkmale im Datensatz sehr unterschiedlich strukturiert sind und nicht alle Prädiktoren gleich behandelt werden sollten. Durch eine gezielte Auswahl von Features und die Anwendung einer selektiven Skalierung kann das Modell stabilisiert werden. Zusätzliche Transformationen könnten die Analyseergebnisse verbessern und den Einfluss einzelner Prädiktoren verstärken.



#### **Ergebnis Vor-Analyse**

Die Entwicklung der Analysestrategie war ein iterativer Prozess, der durch mehrere wichtige Erkenntnisse geprägt wurde. Zu Beginn der Untersuchung wurden alle verfügbaren Prädiktoren einbezogen, wobei sich früh zeigte, dass die am stärksten beeinflussbaren Faktoren wie Thumbnail-Gestaltung und Titelbewertung überraschend geringe Korrelationen mit den Zielvariablen aufwiesen.

Eine genauere Untersuchung der Daten offenbarte, dass diese schwachen Zusammenhänge teilweise durch die besondere Charakteristik von Live-Übertragungen verzerrt wurden. Live-Videos folgen grundsätzlich anderen Gesetzmäßigkeiten als reguläre Nachrichtenvideos: Sie werden oft über Breaking News oder wichtige Ereignisse automatisch als Startseiten-Empfehlung ausgespielt, was zu überdurchschnittlich hohen Klickraten führt - unabhängig von Thumbnail oder Titel. Zudem werden Live-Videos häufig mit standardisierten Thumbnails und Titeln versehen, da bei Breaking News die Zeit für aufwändige Optimierungen fehlt.

**Warum Kategorie "Live" aus den Daten nehmen**

Diese Erkenntnis führte zu dem Entschluss, Live-Videos aus der Hauptanalyse auszuschließen, um die tatsächlichen Effekte der beeinflussbaren Faktoren bei regulären Nachrichtenvideos besser isolieren zu können. Die verbleibenden schwachen bis moderaten Zusammenhänge zwischen Thumbnail-Gestaltung, Titelbewertung und Performance-Metriken deuten darauf hin, dass der Erfolg von YouTube-Videos im Nachrichtenbereich von einem komplexen Zusammenspiel verschiedener Faktoren abhängt, die über rein gestalterische Aspekte hinausgehen.

**Ausreisseranalyse**

Die anschließende Ausreißeranalyse bestätigte diese Vermutung: Erst nach der Bereinigung extremer Ausreißer, die oft durch externe Faktoren wie besondere Nachrichtenereignisse oder algorithmusbedingte Empfehlungsschübe entstehen, wurden die tatsächlichen Effekte der Gestaltungselemente statistisch signifikant nachweisbar. Dies unterstreicht die Notwendigkeit, bei der Analyse von YouTube-Metriken im Nachrichtenbereich sowohl die besonderen Charakteristika verschiedener Videoformate als auch externe Einflussfaktoren zu berücksichtigen.

Diese Erkenntnisse führten zu einem verfeinerten Analyseansatz, der sich auf reguläre Nachrichtenvideos konzentriert und durch geeignete statistische Methoden wie die IQR-basierte Ausreißerbereinigung die subtileren Effekte der Gestaltungselemente besser herausarbeiten kann.

## Methodology

> REMOVE THE FOLLOWING TEXT

This section includes a brief description of your modeling process.

Explain the reasoning for the type of model you're fitting, predictor variables considered for the model.

Additionally, show how you arrived at the final model by describing the model selection process, variable transformations (if needed), assessment of conditions and diagnostics, and any other relevant considerations that were part of the model fitting process.

##### Modellierungsansatz für  Klickrate der Impressions

Im ersten Schritt werde ich mich auf die Klickrate als Zielvariable konzentrieren um erste **statistische Erkenntnisse** über die Variablen und deren Einfluss zu gewinnen. 

---
#### 1. Explorative Datenanalyse (EDA)

- **Ziel**: Erste Muster erkennen und Hypothesen zu potenziellen Einflussfaktoren aufstellen.  
- **Methoden**:  
  - **Boxplots** für Vergleiche zwischen Kategorien (z. B. Themengebiete, Thumbnail-Gestaltung)  
  - **Streudiagramme** für Zusammenhänge mit numerischen Variablen (z. B. Videolänge, SEO-Bewertung)  
  - **Deskriptive Statistiken** (Mittelwert, Median, Standardabweichung) zur Beschreibung der Verteilung  
---
#### 2. Auswahl der Prädiktorvariablen
- Auf Basis der EDA:  
  - **Kategoriale Variablen** wie *Thema* oder *Thumbnail*  
  - **Numerische Variablen** wie *SEO-Bewertung des Titels*  
- **Ziel**: Variablen auswählen, die signifikante Zusammenhänge mit der Klickrate aufweisen.
---
#### 3. Regressionsanalyse

**Lineare Regression**  
   - Einfacher Einstieg, um den linearen Zusammenhang zwischen Prädiktoren und Klickrate zu testen.
---

#### 4. Residuenanalyse

- **Prüfung der Modellannahmen**  
  - Verteilung und Zufälligkeit der Residuen kontrollieren  
- **Identifikation von Mustern**  
  - Hinweise auf fehlende Variablen oder nichtlineare Beziehungen  
---
#### 5. Statistische Tests und Bewertung
- **F-Test**: Überprüfung, ob das Modell insgesamt signifikant ist.  
- **P-Werte**: Bewertung der Relevanz einzelner Prädiktoren.  
- **R²**: Anteil der durch das Modell erklärten Varianz der Klickrate.  
---


## Results

> REMOVE THE FOLLOWING TEXT

This is where you will output the final model with any relevant model fit statistics.

Describe the key results from the model.
The goal is not to interpret every single variable in the model but rather to show that you are proficient in using the model output to address the research questions, using the interpretations to support your conclusions.

Focus on the variables that help you answer the research question and that provide relevant context for the reader.


## Discussion + Conclusion


> REMOVE THE FOLLOWING TEXT

In this section you'll include a summary of what you have learned about your research question along with statistical arguments supporting your conclusions.
In addition, discuss the limitations of your analysis and provide suggestions on ways the analysis could be improved.
Any potential issues pertaining to the reliability and validity of your data and appropriateness of the statistical analysis should also be discussed here.
Lastly, this section will include ideas for future work.